In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string
import nltk
import gensim
import spacy
import pickle
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from gensim.utils import simple_preprocess
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("Data/data.csv")

In [3]:
train.head(15)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5,5,3,1,2,0,1,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just..."
6,6,3,0,3,0,1,"!!!!!!""@__BrighterDays: I can not just sit up ..."
7,7,3,0,3,0,1,!!!!&#8220;@selfiequeenbri: cause I'm tired of...
8,8,3,0,3,0,1,""" &amp; you might not get ya bitch back &amp; ..."
9,9,3,1,2,0,1,""" @rhythmixx_ :hobbies include: fighting Maria..."


In [4]:
print(len(train))

24783


In [5]:
train.groupby('class').nunique()

,Unnamed: 0,count,hate_speech,offensive_language,neither,tweet
class,,,,,,
0,1430,5,6,5,5,1430
1,19190,5,5,8,4,19190
2,4163,5,4,5,8,4163


In [6]:
train = train[['tweet','class']]
train.head()

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [7]:
train['tweet'].isnull().sum()

0

In [8]:
# Data processing
def depure_data(data):
    #Removing URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'',data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)

    return data

In [9]:
temp = []
# Splitting pd.Series to data to list
data_to_list = train["tweet"].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
print(list(temp[:5]))

['!!! RT As a woman you shouldnt complain about cleaning up your house. &amp; as a man you should always take the trash out...', '!!!!! RT boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!', '!!!!!!! RT Dawg!!!! RT You ever fuck a bitch and she start to cry? You be confused as shit', '!!!!!!!!! RT she look like a tranny', '!!!!!!!!!!!!! RT The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;']


In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [11]:
data_words = list(sent_to_words(temp))
print(data_words[:10])
print(len(data_words))

[['rt', 'as', 'woman', 'you', 'shouldnt', 'complain', 'about', 'cleaning', 'up', 'your', 'house', 'amp', 'as', 'man', 'you', 'should', 'always', 'take', 'the', 'trash', 'out'], ['rt', 'boy', 'dats', 'cold', 'tyga', 'dwn', 'bad', 'for', 'cuffin', 'dat', 'hoe', 'in', 'the', 'st', 'place'], ['rt', 'dawg', 'rt', 'you', 'ever', 'fuck', 'bitch', 'and', 'she', 'start', 'to', 'cry', 'you', 'be', 'confused', 'as', 'shit'], ['rt', 'she', 'look', 'like', 'tranny'], ['rt', 'the', 'shit', 'you', 'hear', 'about', 'me', 'might', 'be', 'true', 'or', 'it', 'might', 'be', 'faker', 'than', 'the', 'bitch', 'who', 'told', 'it', 'to', 'ya'], ['the', 'shit', 'just', 'blows', 'me', 'claim', 'you', 'so', 'faithful', 'and', 'down', 'for', 'somebody', 'but', 'still', 'fucking', 'with', 'hoes'], ['can', 'not', 'just', 'sit', 'up', 'and', 'hate', 'on', 'another', 'bitch', 'got', 'too', 'much', 'shit', 'going', 'on'], ['cause', 'im', 'tired', 'of', 'you', 'big', 'bitches', 'coming', 'for', 'us', 'skinny', 'girls'],

In [12]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [13]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

['rt as woman you shouldnt complain about cleaning up your house amp as man you should always take the trash out', 'rt boy dats cold tyga dwn bad for cuffin dat hoe in the st place', 'rt dawg rt you ever fuck bitch and she start to cry you be confused as shit', 'rt she look like tranny', 'rt the shit you hear about me might be true or it might be faker than the bitch who told it to ya']


In [14]:
data = np.array(data)

In [15]:
# Labels
labels = np.array(train["class"])
labels = tf.keras.utils.to_categorical(labels,3,dtype="int32")
print(len(labels))

24783


In [16]:
# Data Sequencing and Splitting
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

In [17]:
max_words = 5000 # Consider only the top 5k words
max_len = 200 # Consider only the 200 first word of each tweet

tokenizer3 = Tokenizer(num_words=max_words)
tokenizer3.fit_on_texts(data)
sequences = tokenizer3.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
with open('tockenizer3.pickle','wb') as handle:
    pickle.dump(tokenizer3,handle,protocol=pickle.HIGHEST_PROTOCOL)

print(tweets)
print(labels)

[[   0    0    0 ...    3   47   52]
 [   0    0    0 ...    3  543  515]
 [   0    0    0 ...  993   71   41]
 ...
 [   0    0    0 ...  341   28  274]
 [   0    0    0 ... 1826    4 1307]
 [   0    0    0 ...  171   55  115]]
[[0 0 1]
 [0 1 0]
 [0 1 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 0 1]]


In [18]:
x_train, x_test, y_train, y_test = train_test_split(tweets,labels,random_state=0)
print(len(x_train),len(x_test),len(y_train),len(y_test))

18587 6196 18587 6196


## Build and Train NN

In [19]:
# BiDR LST LAYER MODEL
model = Sequential([
    layers.Embedding(max_words,128,input_length=max_len),
    layers.Bidirectional(layers.LSTM(64,return_sequences=True)),
    layers.Bidirectional(layers.LSTM(64)),
    layers.Dense(3,activation="softmax"),
])
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])
checkpoint = ModelCheckpoint("Model/best_model3.hdf5",monitor="val_accuracy",verbose=1,save_best_only=True,mode='auto',period=1,save_weights_only=False)
history = model.fit(x_train, y_train, epochs=4, validation_data=(x_test,y_test), verbose=2, callbacks=[checkpoint])

Epoch 1/4
581/581 - 24s - loss: 0.3725 - accuracy: 0.8728 - val_loss: 0.2984 - val_accuracy: 0.8975

Epoch 00001: val_accuracy improved from -inf to 0.89751, saving model to Model/best_model3.hdf5
Epoch 2/4
581/581 - 19s - loss: 0.2661 - accuracy: 0.9095 - val_loss: 0.2716 - val_accuracy: 0.9049

Epoch 00002: val_accuracy improved from 0.89751 to 0.90494, saving model to Model/best_model3.hdf5
Epoch 3/4
581/581 - 19s - loss: 0.2386 - accuracy: 0.9185 - val_loss: 0.2737 - val_accuracy: 0.9040

Epoch 00003: val_accuracy did not improve from 0.90494
Epoch 4/4
581/581 - 19s - loss: 0.2213 - accuracy: 0.9254 - val_loss: 0.2707 - val_accuracy: 0.9074

Epoch 00004: val_accuracy improved from 0.90494 to 0.90736, saving model to Model/best_model3.hdf5


## Test the model on dataset X_test

In [20]:
best_model = tf.keras.models.load_model("Model/best_model3.hdf5")
test_loss, test_acc = best_model.evaluate(x_test,y_test,verbose=2)
print("Test accuracy: {:.2f}%".format(100*test_acc))

194/194 - 3s - loss: 0.2707 - accuracy: 0.9074
Test accuracy: 90.74%
